# STEP49 Datasetクラスと前処理

## このSTEPの概要

### STEP48で学んだこととその課題

- スパイラルデータセットを使って多値分類をおこなった。
    - `x, t = dezero.datasets.get_spiral()` というコードでテータの読み込みを行った
        - `x`: (300, 2), ndarray
        - `t`: (300,), ndarray
- しかしHWが保持しているメモリより大規模なデータセットを扱う場合、すべてのデータをこのようにndarrayに読むことができない課題がある
- また前処理に関して標準化された仕様になっていない。

### STEP49の目的

上記のような問題に対応できるようにデータセット専用のクラス `Dataset` クラスを作る。更に前処理を行える仕組みも入れる

- 49.1 Datasetクラスの実装
    - `dezero/datasets.py` に`Dataset` クラスを実装
    - `dezero/datasets.py` に`Spiral`クラスを実装
- 49.2 大きいデータセットの場合
    - `dezero/datasets.py` に `BigData` クラスを実装
- 49.3 データの連結
    - インデックス操作によりミニバッチに対応する複数データを取り出す
        - [3, 5, 8, 2, 1] => Indexがシャフルされ、その値が抽出される。。。。
- 49.4 学習用のコード
    - DatasetクラスによりSpiralデータセットのコードがSTEP48と比較してどう変わるか確認する
- 49.5 データセットの前処理
    - 前処理およびデータ拡張を対応可能にするために、ユーザが定義した関数を関数オブジェクトで引数で渡せるようにする

### Datasetクラスで必要な仕様は以下の２つである

いったん以下を頭に入れておけば良い

- Dataset クラスで必要な条件は以下の２つのメソッドを含んでいること
    - `__getitem__`: Pythonの特殊メソッドx[0], x[1]などカッコつきでアクセスしたときの定義
    - `__len__`: 例えばlen(x)を使ったときに呼ばれる特殊メソッド

これまで何をやったか、まとめています。目次より、少し詳しい情報となっています。

https://qiita.com/daikumatan/private/1c4ba888ed1928c55fb2

## 事前準備

In [65]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import math
import numpy as np
import matplotlib.pyplot as plt
import dezero
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP

## 49.1 Datasetクラスの実装

49.1では前述のHWのメモリ量を超えるような、大きいデータはまだ扱えないことに注意する

### `dezero/datasets.py` に`Dataset` クラスを実装する

- コンストラクタ
    - 引数 `train`: 学習用とテスト用を区別するためのフラグ
    - インスタンス変数 `data`: 入力データを保持する
    - インスタンス変数 `label`: ラベルデータを保持する
- `__getitem__` メソッド
    - pythonの特殊メソッドであり `x[0]`, `x[1]` など、角括弧[]でアクセスしたときの挙動を定義する
    - 単に、Datasetクラスの指定されたインデックスのデータを取りだす
    - `self.data[index]` で指定されたとき、ラベルにデータが無いときは、`None` を返却する
    - スライス機能 (例えば `x[1:3]`) には対応しない
- `__len__` メソッド
    - `len()`関数を使ったときに呼ばれる

```python
class Dataset:
    def __init__(self, train=True):
        self.train = train
        self.data = None     # 教師データ
        self.label = None    # ラベルデータ
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)  # スカラのみ対応
        if self.label is None:
            return self.data[index], None
        else:
            return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

    def prepare(self): # 継承先で具体的動作を規定
        pass
```

### `dezero/datasets.py` に`Spiral`クラスを実装する

- `Dataset` クラスを継承し、`prepare` メソッドをオーバーライドして実装する
    - `get_spriral`は以前実装し説明したため省略

```python
class Spiral(Dataset):
    def prepare(self):
        self.data, self.label = get_spiral(self.train)
```

### Spiralクラスのデータ取り出し例

- index:0のDataとラベルがタプルとして返却されている

In [66]:
train_set = dezero.datasets.Spiral(train=True)
print(train_set[0])
print(len(train_set))

(array([-0.13981389, -0.00721657], dtype=float32), 1)
300


## 49.2 大きいデータセットの場合

### `BigData()`クラスの実装例

ここでは以下のディレクトリがあることを想定

- data/ (100万個のデータが保存)
- labe/ (100万個のラベルが保存)

データが大きいときは`Dataset`クラスのインスタンス変数 `data`, `label` に直接ndarrayインスタンスを保持できない。  
そのためBigDataクラスの初期化時には、それらのデータは読み込まず、データへのアクセスがあったタイミングで読み込むようにする。

具体的には以下のステップを踏む

- `__getitem__(index)` が呼ばれるタイミングで、dataディレクトリーにあるデータを読み込む
- `np.load` に関してはSTEP53で説明

```python
class BigData(Dataset):
    def __getitem__(self, index):
        # indexで指定されたファイルを読み込んでいる。
        # このファイルは100万個のデータがそれぞれ保存されている
        # つまりイメージとしては100万個セットで1つファイルが複数あり、それをIndexで指定するイメージ
        x = np.load('data/{}.npy'.format(index))
        t = np.load('label/{}.npy'.format(index))
        return x, t
    
    def __len__():
        return 1000000
```

## 49.3 データの連結

### ミニバッチとして取り出すコード

- インデックス操作により、ミニバッチに対応する複数データを取り出す
- 下記のようなデータとラベルが1セットのデータが取り出せていることを確認できる
    - `[(array([-0.13981389, -0.00721657], dtype=float32), 1), ...]`
    - ここで、
        - `[-0.13981389, -0.00721657]`: Trainデータであり スパイラルデータセットの(x, y)座標
        - `1`: ラベルデータ

In [67]:
train_set = dezero.datasets.Spiral()
batch_index = [0, 1, 2]
batch = [train_set[i] for i in batch_index]
batch

[(array([-0.13981389, -0.00721657], dtype=float32), 1),
 (array([0.37049392, 0.5820947 ], dtype=float32), 1),
 (array([ 0.1374263 , -0.17179643], dtype=float32), 2)]

### Dezeroの入力データとして利用するために、取り出したデータを`ndarray`インスタンスに変換する

In [68]:
x = np.array([example[0] for example in batch])
t = np.array([example[1] for example in batch])

print(x, x.shape)
print(t, t.shape)

[[-0.13981389 -0.00721657]
 [ 0.37049392  0.5820947 ]
 [ 0.1374263  -0.17179643]] (3, 2)
[1 1 2] (3,)


## 49.4 学習用のコード

### STEP48からの変更点

- Spiralクラスを使うようにした。
- それに伴い、ミニバッチを形成するコードを以下のように変更した

#### 変更前(STEP48)

```python
        :
x, t = dezero.datasets.get_spiral(train=True)
        :
    for i in range(max_iter):
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch_x = x[batch_index]
        batch_t = t[batch_index]
```

#### 変更後(STEP4９)

```python
        :
train_set = dezero.datasets.Spiral(train=True)
        :
    for i in range(max_iter):
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch_x = np.array([example[0] for example in batch])  # data
        batch_t = np.array([example[1] for example in batch])  # label
```

### 変更によるご利益

- もし、データサイズが大きくなれば、SpiralクラスをBigdataクラスに変更すれば良い。
- つまり簡単にデータセットの入れ替えができるようになった

In [69]:
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0

# データ読み込み
train_set = dezero.datasets.Spiral(train=True)
# 隠れ層: 10nodes, 出力層: 3nodes
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

# 以下の2行で1エポックあたりのステップ数を計算
data_size = len(train_set)
max_iter = math.ceil(data_size / batch_size)

for epoch in range(max_epoch):
    # Shuffle index for data,
    # 300あるデータのうち、idを並び替えてシャフルするエポックのループのたびにシャフルされることに注意
    #index = np.random.permutation(data_size)

    # index のシャフルのオリジナルは上記だがコメントアウトしている2行のほうが高速で現在はこちらを使うべきだろう
    rng = np.random.default_rng()
    index = rng.permutation(data_size)
    sum_loss = 0

    # max_iterはデータサイズ/バッチサイズなので10となる。
    for i in range(max_iter):
        # Create minibatch, # シャフルされたデータから、バッチサイズ分データを切り取る
        batch_start = i * batch_size
        batch_end   = (i + 1) * batch_size
        batch_index = index[batch_start:batch_end]
        batch = [train_set[i] for i in batch_index]            # [変更点]
        batch_x = np.array([example[0] for example in batch])  # [変更点]　data
        batch_t = np.array([example[1] for example in batch])  # [変更点]　label

        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()      # ループが回るので、いったんCleanUpが必要
        loss.backward()
        optimizer.update()

        # loss.dataはスカラー値, バッチ処理数分の長さを書ける
        # loss.dataは30データ分まとまった、平均値として出力される。
        # 最後にdata_sizeで割って1データあたりのavg_lossを求めるので、
        # いったんlen(batch_t) = 30 をかけている
        sum_loss += float(loss.data) * len(batch_t)

    # Print loss every epoch
    avg_loss = sum_loss / data_size
    print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))


epoch 1, loss 1.00
epoch 2, loss 0.94
epoch 3, loss 0.88
epoch 4, loss 0.84
epoch 5, loss 0.85
epoch 6, loss 0.82
epoch 7, loss 0.80
epoch 8, loss 0.78
epoch 9, loss 0.76
epoch 10, loss 0.80
epoch 11, loss 0.78
epoch 12, loss 0.78
epoch 13, loss 0.78
epoch 14, loss 0.78
epoch 15, loss 0.76
epoch 16, loss 0.77
epoch 17, loss 0.76
epoch 18, loss 0.78
epoch 19, loss 0.75
epoch 20, loss 0.75
epoch 21, loss 0.77
epoch 22, loss 0.76
epoch 23, loss 0.75
epoch 24, loss 0.75
epoch 25, loss 0.75
epoch 26, loss 0.76
epoch 27, loss 0.74
epoch 28, loss 0.74
epoch 29, loss 0.72
epoch 30, loss 0.75
epoch 31, loss 0.75
epoch 32, loss 0.74
epoch 33, loss 0.72
epoch 34, loss 0.72
epoch 35, loss 0.74
epoch 36, loss 0.71
epoch 37, loss 0.71
epoch 38, loss 0.73
epoch 39, loss 0.70
epoch 40, loss 0.73
epoch 41, loss 0.70
epoch 42, loss 0.69
epoch 43, loss 0.70
epoch 44, loss 0.69
epoch 45, loss 0.72
epoch 46, loss 0.71
epoch 47, loss 0.69
epoch 48, loss 0.67
epoch 49, loss 0.67
epoch 50, loss 0.67
epoch 51,

## 49.5 データセットの前処理

### 前処理の例

- データからある値を差し引く
- データの形状を変形する
- データ拡張処理 (データを増やす)
    - 画像を回転させる
    - 画像を左右反転させる

### `Dataset()` クラスの改善

- 前処理およびデータ拡張を対応可能にするために、ユーザが定義した関数を関数オブジェクトで引数で渡せるようにする
- コンストラクタの引数説明
    - `transform`: 学習データ用の前処理関数オブジェクトが引数として渡される
        - 指定されなかったとき(Default値)
            - `self.transform = lambda x: x` が実行される
            - これは値をそのまま渡すだけの関数(つまり何もしない)
    - `target_transform`: ラベル用の前処理関数オブジェクトが引数として渡される 
        - 指定されなかったとき(Default値)
            - `self.target_transform = lambda x: x` が実行
            - これは値をそのまま渡すだけの関数(つまり何もしない)

```python
class Dataset:
    def __init__(self, train=True, transform=None, target_transform=None):   # 変更
        self.train = train                           
        self.transform = transform                  # 追記  
        self.target_transform = target_transform    # 追記
        if self.transform is None:                  # 追記: 引数に何も設定されなかったとき
            self.transform = lambda x: x            # 追記: 前処理なしに値をそのまま返す関数を設定
        if self.target_transform is None:           # 追記: 引数に何も設定されなかったとき
            self.target_transform = lambda x: x     # 追記: 前処理なしに値をそのまま返す関数を設定

        self.data = None
        self.label = None
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)
        if self.label is None:
            return self.transform(self.data[index]), None   # 変更: 前処理関数を実行, 学習データのみ
        else:
            return self.transform(self.data[index]),\
                   self.target_transform(self.label[index]) # 変更: 前処理関数を実行, 学習データ, ラベルに対して実施

    def __len__(self):
        return len(self.data)

    def prepare(self):
        pass
```

### 使用例: 1/2にスケール変換する前処理

- 1/2スケールを実現する関数`f`を定義し、その関数オブジェクトを `transform=f` として渡す

In [70]:
# 1/2を実現する関数
def f(x):
    y = x / 2.0
    return y

# 関数オブジェクトを渡す
train_set = dezero.datasets.Spiral(transform=f)
#train_set = dezero.datasets.Spiral()

# 1/2になったことを確認する
for val in train_set:
    print(val[0], val[1])

[-0.06990694 -0.00360829] 1
[0.18524696 0.29104736] 1
[ 0.06871315 -0.08589821] 2
[0.1515844 0.03236  ] 0
[-0.10424428  0.26525107] 1
[-0.35371885 -0.066955  ] 2
[ 0.24727833 -0.18695834] 0
[0.11600986 0.06904139] 0
[-0.07943024 -0.00953086] 1
[ 0.00245854 -0.16998222] 2
[-0.06497979 -0.00162078] 1
[-0.11987292  0.12749699] 1
[-0.16605952 -0.21289489] 2
[-0.07203399  0.32716373] 1
[-0.04386805  0.24104065] 1
[-0.42339772  0.17981759] 2
[0.17496437 0.30312946] 1
[-0.13280705 -0.19990571] 2
[ 0.01936758 -0.14370072] 2
[-0.08994609  0.0031146 ] 1
[-0.13261119 -0.22943251] 2
[-0.00793436  0.2698834 ] 1
[0.07531573 0.0934213 ] 0
[-0.3890553   0.02712853] 2
[-0.05284092 -0.01525901] 1
[0.08585732 0.09084889] 0
[-0.02530199 -0.213506  ] 2
[-0.2939061  -0.11333232] 2
[ 0.21367101 -0.05238989] 0
[0.00999704 0.00024344] 2
[-0.32779983 -0.32271856] 0
[0.04666041 0.05871802] 0
[-0.02338564 -0.00883808] 1
[-0.29350457 -0.39859763] 0
[0.09493664 0.05556109] 0
[-0.13748518 -0.20278764] 2
[-0.3313409 

### 良く行われる前処理を `dezero/transforms.py` に実装

DeZeroではよく使われる前処理を`dezero/transforms.py`に用意している

- データの正規化
- 画像データに関する変換処理

### 正規化の例

ここでは正規化の例を調べてみる

- `(x - mean)/std` の処理が行われる
- 以下のコードがコールされる

```python
class Normalize:
    """Normalize a NumPy array with mean and standard deviation.

    Args:
        mean (float or sequence): mean for all values or sequence of means for
         each channel.
        std (float or sequence):
    """
    def __init__(self, mean=0, std=1):
        self.mean = mean
        self.std = std

    def __call__(self, array):
        mean, std = self.mean, self.std

        if not np.isscalar(mean):
            mshape = [1] * array.ndim
            mshape[0] = len(array) if len(self.mean) == 1 else len(self.mean)
            mean = np.array(self.mean, dtype=array.dtype).reshape(*mshape)
        if not np.isscalar(std):
            rshape = [1] * array.ndim
            rshape[0] = len(array) if len(self.std) == 1 else len(self.std)
            std = np.array(self.std, dtype=array.dtype).reshape(*rshape)
        return (array - mean) / std
```

In [71]:
from dezero import transforms

f = transforms.Normalize(mean=0.0, std=1.0)
train_set = dezero.datasets.Spiral(transform=f)
for val in train_set:
    print(val[0], val[1])


[-0.13981389 -0.00721657] 1
[0.37049392 0.5820947 ] 1
[ 0.1374263  -0.17179643] 2
[0.3031688 0.06472  ] 0
[-0.20848857  0.53050214] 1
[-0.7074377 -0.13391  ] 2
[ 0.49455667 -0.3739167 ] 0
[0.23201972 0.13808277] 0
[-0.15886047 -0.01906173] 1
[ 0.00491708 -0.33996445] 2
[-0.12995958 -0.00324155] 1
[-0.23974584  0.25499398] 1
[-0.33211905 -0.42578977] 2
[-0.14406797  0.65432745] 1
[-0.08773611  0.4820813 ] 1
[-0.84679544  0.35963517] 2
[0.34992874 0.6062589 ] 1
[-0.2656141  -0.39981142] 2
[ 0.03873516 -0.28740144] 2
[-0.17989218  0.00622921] 1
[-0.26522237 -0.45886502] 2
[-0.01586872  0.5397668 ] 1
[0.15063146 0.1868426 ] 0
[-0.7781106   0.05425706] 2
[-0.10568184 -0.03051801] 1
[0.17171463 0.18169777] 0
[-0.05060399 -0.427012  ] 2
[-0.5878122  -0.22666465] 2
[ 0.42734203 -0.10477977] 0
[0.01999407 0.00048688] 2
[-0.65559965 -0.6454371 ] 0
[0.09332082 0.11743604] 0
[-0.04677129 -0.01767616] 1
[-0.58700913 -0.79719526] 0
[0.18987328 0.11112218] 0
[-0.27497035 -0.40557528] 2
[-0.6626818  -

### 複数の変換処理を続けて行う場合

- `Compose` クラスを利用する
    - 与えられた処理を先頭から順に処理する
- この例では、正規化を行い、float64に変換する処理を行う 

```python
class Compose:
    """Compose several transforms.

    Args:
        transforms (list): list of transforms
    """
    def __init__(self, transforms=[]):
        self.transforms = transforms

    # Datasetクラスから, 実際のdata, ここでは(x ,y)座標のデータが引数に入る
    def __call__(self, img):
        if not self.transforms:    # リストが空のとき
            return img
        # 関数の引数にdataを引数に与えて、imgという変数を更新している
        for t in self.transforms:
            img = t(img)
        return img
```

In [72]:
f = transforms.Compose([transforms.Normalize(mean=0.0, std=2.0), 
                        transforms.AsType(np.float64)])
train_set = dezero.datasets.Spiral(transform=f)

In [73]:
for val in train_set:
    print(val[0], val[1])

[-0.06990694 -0.00360829] 1
[0.18524696 0.29104736] 1
[ 0.06871315 -0.08589821] 2
[0.1515844 0.03236  ] 0
[-0.10424428  0.26525107] 1
[-0.35371885 -0.066955  ] 2
[ 0.24727833 -0.18695834] 0
[0.11600986 0.06904139] 0
[-0.07943024 -0.00953086] 1
[ 0.00245854 -0.16998222] 2
[-0.06497979 -0.00162078] 1
[-0.11987292  0.12749699] 1
[-0.16605952 -0.21289489] 2
[-0.07203399  0.32716373] 1
[-0.04386805  0.24104065] 1
[-0.42339772  0.17981759] 2
[0.17496437 0.30312946] 1
[-0.13280705 -0.19990571] 2
[ 0.01936758 -0.14370072] 2
[-0.08994609  0.0031146 ] 1
[-0.13261119 -0.22943251] 2
[-0.00793436  0.26988339] 1
[0.07531573 0.0934213 ] 0
[-0.38905531  0.02712853] 2
[-0.05284092 -0.01525901] 1
[0.08585732 0.09084889] 0
[-0.02530199 -0.213506  ] 2
[-0.29390609 -0.11333232] 2
[ 0.21367101 -0.05238989] 0
[0.00999704 0.00024344] 2
[-0.32779983 -0.32271856] 0
[0.04666041 0.05871802] 0
[-0.02338564 -0.00883808] 1
[-0.29350457 -0.39859763] 0
[0.09493664 0.05556109] 0
[-0.13748518 -0.20278764] 2
[-0.33134091

## 補足

### Softmax関数の復習

$$
p_k = \frac{\exp\left(y_k\right)}{\sum_{i=1}^n \exp\left(y_i\right)}
$$

このとき, 
$$
p_1 + p_2 + \cdot \cdot \cdot + p_n = 1
$$

- Softmax関数は定義式からわかるように指数関数の計算を行うため値が発散しがちになる
- そのため、Softmax関数ではオーバーフロー対策を行うのが一般的(本書では説明しない)